# Motivation

The emergence and ubiquitousness of the social media like Facebook, Twitter and Instagram have given each individual a platform to connect with people and also speak for himself/herself. People are becoming more and more used to express their thoughts though social media which have already become a "We media". People are also affected by other people's comments and opinions, which could be heard by more through social media. Interesingly also, one could get a sense of whether a certain product, organization or people is liked or not by monitoring social media/forums. In fact, Companies are already taking steps of managing branding or reputation over the internet.

In the mean time, the widely use of social medias have also provided with researchers a large amount of data and interesting topics to dive into, for example social network analysis. Twitter, in particular, being a platform where people make comments, produces tons of natural language data everyday. Utilizing NLP and machine learning algorithms, we could do many intersting things with the tweets people posted. In this project, we are interested in analyzing the reputation of certain entities by sentiment analysis on tweets.

# Approach



# Data

# Code

# Experimental Setup

# Results

# Analysis of the Results

# Future Work